In [37]:
import chess
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from collections import deque
import random

In [39]:
# --------------------------
# 1. Định nghĩa Môi trường
# --------------------------
class ChessEnv:
    def __init__(self):
        self.board = chess.Board()
    
    def reset(self):
        """Reset bàn cờ về trạng thái ban đầu"""
        self.board.reset()
        return self.get_state()
    
    def get_state(self):
        """Chuyển bàn cờ sang tensor 8x8x14"""
        # 14 kênh: 6 loại quân * 2 màu + 2 kênh meta (nước đi, nhập thành)
        state = np.zeros((8, 8, 14), dtype=np.float32)
        
        for square in chess.SQUARES:
            piece = self.board.piece_at(square)
            if piece:
                row, col = chess.square_rank(square), chess.square_file(square)
                # Kênh 0-5: Quân trắng (PAWN, KNIGHT,..., KING)
                # Kênh 6-11: Quân đen
                channel = piece.piece_type - 1 + (6 if piece.color == chess.BLACK else 0)
                state[row, col, channel] = 1
                
        # Kênh 12: Số nước đi (chuẩn hóa)
        state[:, :, 12] = self.board.fullmove_number / 100.0
        # Kênh 13: Quyền nhập thành
        state[:, :, 13] = self.board.has_castling_rights(chess.WHITE) | self.board.has_castling_rights(chess.BLACK)
        return state
    
    def get_legal_moves(self):
        """Trả về danh sách các nước đi hợp lệ dưới dạng chỉ số"""
        return [self.move_to_index(m) for m in self.board.legal_moves]
    
    def move_to_index(self, move):
        """Chuyển đổi chess.Move thành index duy nhất (0-4671)"""
        return move.from_square * 64 + move.to_square
    
    def index_to_move(self, index):
        """Chuyển index thành chess.Move"""
        from_sq = index // 64
        to_sq = index % 64
        return chess.Move(from_sq, to_sq)
    
    def step(self, action_index):
        """Thực hiện nước đi và trả về (next_state, reward, done)"""
        move = self.index_to_move(action_index)
        self.board.push(move)
        
        reward = 0
        done = self.board.is_game_over()
        
        # Tính reward
        if done:
            if self.board.is_checkmate():
                reward = 1 if self.board.turn == chess.BLACK else -1  # Đối phương vừa chiếu hết
            else:  # Hòa
                reward = 0
        return self.get_state(), reward, done

In [41]:

# Cài đặt thuật toán negamax với cắt tỉa alpha-beta
def negamax_alpha_beta(board: chess.Board, depth: int, alpha: float, beta: float, color: int) -> float:
    if depth == 0 or board.is_game_over():
        return color * evaluate_board(board)

    max_value = -math.inf
    # Duyệt tất cả các nước đi hợp lệ
    for move in board.legal_moves:
        board.push(move)
        # Sử dụng đệ quy với lượt đi đối nghịch (-color)
        value = -negamax_alpha_beta(board, depth - 1, -beta, -alpha, -color)
        board.pop()
        
        max_value = max(max_value, value)
        alpha = max(alpha, value)
        # Nếu alpha >= beta => cắt tỉa (pruning)
        if alpha >= beta:
            break
    return max_value

In [43]:
# --------------------------
# 2. Định nghĩa Neural Network
# --------------------------
class ChessPolicyNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # Convolutional layers
        self.conv = nn.Sequential(
            nn.Conv2d(14, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU()
        )
        
        # Policy head (dự đoán xác suất nước đi)
        self.policy_head = nn.Sequential(
            nn.Linear(256 * 8 * 8, 512),
            nn.ReLU(),
            nn.Linear(512, 4672)  # 64*64 = 4096 possible moves (thực tế ít hơn)
        )
        
        # Value head (dự đoán giá trị state)
        self.value_head = nn.Sequential(
            nn.Linear(256 * 8 * 8, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Tanh()
        )
    
    def forward(self, x):
        x = self.conv(x)
        x = x.reshape(x.size(0), -1)
        policy = self.policy_head(x)
        value = self.value_head(x)
        return policy, value

In [45]:
# --------------------------
# 3. Định nghĩa Agent
# --------------------------
class PPOChessAgent:
    def __init__(self, lr=1e-4, gamma=0.99, clip_epsilon=0.2):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.policy_net = ChessPolicyNetwork().to(self.device)
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=lr)
        self.gamma = gamma  # Discount factor
        self.clip_epsilon = clip_epsilon
        self.memory = deque(maxlen=10000)  # Experience replay
    
    def select_action(self, state, legal_moves):
        """Chọn action dựa trên policy network và các nước đi hợp lệ"""
        state_tensor = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(self.device)
        state_tensor = state_tensor.permute(0, 3, 1, 2)
        
        with torch.no_grad():
            logits, _ = self.policy_net(state_tensor)
            probs = torch.softmax(logits, dim=1).squeeze().cpu().numpy()
        
        # Lọc chỉ các nước đi hợp lệ
        valid_probs = np.zeros_like(probs)
        valid_probs[legal_moves] = probs[legal_moves]
        valid_probs /= valid_probs.sum()
        
        action = np.random.choice(len(probs), p=valid_probs)
        return action
    
    def store_experience(self, experience):
        """Lưu experience vào bộ nhớ"""
        self.memory.append(experience)
    
    def train(self, batch_size=128, epochs=4):
        """Huấn luyện trên batch experience"""
        if len(self.memory) < batch_size:
            return
        
        # Lấy ngẫu nhiên một batch từ memory
        batch = random.sample(self.memory, batch_size)
        states, actions, old_probs, rewards, next_states, dones = zip(*batch)
        
        # Chuyển đổi sang tensor
        states = torch.tensor(np.array(states), dtype=torch.float32).to(self.device)
        states = states.permute(0, 3, 1, 2)
        actions = torch.tensor(actions, dtype=torch.long).to(self.device)
        old_probs = torch.tensor(old_probs, dtype=torch.float32).to(self.device)
        
        # Tính discounted rewards
        discounted_rewards = []
        cumulative_reward = 0
        for reward, done in zip(reversed(rewards), reversed(dones)):
            if done:
                cumulative_reward = 0
            cumulative_reward = reward + self.gamma * cumulative_reward
            discounted_rewards.insert(0, cumulative_reward)
        discounted_rewards = torch.tensor(discounted_rewards, dtype=torch.float32).to(self.device)
        
        # Normalize rewards
        discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-7)
        
        for _ in range(epochs):
            # Tính new probabilities
            logits, values = self.policy_net(states)
            new_probs = torch.softmax(logits, dim=1)
            new_probs = new_probs.gather(1, actions.unsqueeze(1)).squeeze()
            
            # Tính tỉ lệ probability
            ratios = new_probs / old_probs
            
            # Tính loss PPO
            surr1 = ratios * discounted_rewards
            surr2 = torch.clamp(ratios, 1 - self.clip_epsilon, 1 + self.clip_epsilon) * discounted_rewards
            policy_loss = -torch.min(surr1, surr2).mean()
            
            # Value loss
            value_loss = nn.MSELoss()(values.squeeze(), discounted_rewards)
            
            # Tổng loss
            total_loss = policy_loss + 0.5 * value_loss
            
            # Backpropagation
            self.optimizer.zero_grad()
            total_loss.backward()
            self.optimizer.step()

In [47]:
# --------------------------
# 4. Huấn luyện
# --------------------------
if __name__ == "__main__":
    env = ChessEnv()
    agent = PPOChessAgent(lr=1e-5)
    
    num_episodes = 1000  # Số game huấn luyện
    batch_size = 128
    
    for episode in tqdm(range(num_episodes)):
        state = env.reset()
        done = False
        episode_reward = 0
        
        while not done:
            # Lấy các nước đi hợp lệ
            legal_moves = env.get_legal_moves()
            
            # Chọn action
            action = agent.select_action(state, legal_moves)
            
            # Thực hiện action
            next_state, reward, done = env.step(action)
            
            # Lưu experience
            with torch.no_grad():
                logits, _ = agent.policy_net(torch.tensor(state).unsqueeze(0).to(agent.device))
                old_prob = torch.softmax(logits, dim=1).squeeze()[action].item()
            
            agent.store_experience((state, action, old_prob, reward, next_state, done))
            
            state = next_state
            episode_reward += reward
            
            # Train định kỳ
            if len(agent.memory) >= batch_size:
                agent.train(batch_size)
        
        # Logging
        if episode % 10 == 0:
            print(f"Episode {episode}, Reward: {episode_reward:.2f}")

    # Lưu model
    torch.save(agent.policy_net.state_dict(), "chess_ppo.pth")

  0%|          | 0/1000 [00:00<?, ?it/s]


RuntimeError: Given groups=1, weight of size [64, 14, 3, 3], expected input[1, 8, 8, 14] to have 14 channels, but got 8 channels instead